In [13]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [17]:
tf.executing_eagerly()

True

In [18]:
c = tf.constant([1, 2, 3])


In [22]:
print(c*c)

tf.Tensor([1 4 9], shape=(3,), dtype=int32)


In [34]:
w = tf.Variable([[1, 1, 1], 
                 [1, 1, 1], 
                 [1, 1, 1]], dtype=tf.float32, shape=(3,3))
x = tf.Variable([[1, 1, 1]], dtype=tf.float32, shape=(1, 3))
with tf.GradientTape() as t:
    y = tf.matmul(tf.matmul(x, w), w)
grad = t.gradient(y, [x, w])
print("y:", y)
print("g(y):", grad)

y: tf.Tensor([[9. 9. 9.]], shape=(1, 3), dtype=float32)
g(y): [<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[9., 9., 9.]], dtype=float32)>, <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[6., 6., 6.],
       [6., 6., 6.],
       [6., 6., 6.]], dtype=float32)>]


In [59]:
len(mnist_img)

60000

In [197]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu',
                          input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(16, [3, 3], activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [192]:
for images, labels in dataset.take(1):
#     print(images)
    print("Logists:", mnist_model(images[0:1]).numpy())

Logists: [[ 0.0024545  -0.05861894  0.00025997  0.01269714 -0.02134838  0.02878507
  -0.01323671 -0.04055608  0.07293171 -0.04307964]]


In [67]:
def loss(predicate, target):
    l = tf.constant(0.001)
    
    

optimizer = tf.keras.optimizers.Adam()


In [172]:
class MSE_WithReg(tf.keras.losses.Loss):
    def __init__(self, model, l):
        self.model = model
        self.l = l
    
    def call(self, y_true, y_pred):
        degree = tf.constant(2, dtype=tf.float32)
        error =tf.reduce_mean(tf.pow(y_true - y_pred, degree, name="pow2"), name="mean")
        return error + self.l*self.reg()
    
    def reg(self):
        s = tf.Variable(0, dtype=tf.float32)
        c = tf.Variable(0, dtype=tf.float32)
        for w in self.model.weights:
            s.assign_add(tf.reduce_mean(w))
            c.assign_add(1)
        s.assign(s/c)
        return s
    @property
    def _name_scope(self):
        return "MSE_WithReg"

In [173]:
loss_mse = MSE_WithReg(mnist_model, tf.constant(0.01))

In [174]:
loss_mse._name_scope

'MSE_WithReg'

In [175]:
y_true = tf.constant([[1, 0, 0, 0,0, 0, 0, 0, 0, 0]], dtype=tf.float32)
for images, labels in dataset.take(1):
    y_pred = mnist_model(images[0:1]).numpy()
    print("loss:", loss_mse.call(y_true, y_pred))

loss: tf.Tensor(0.08989098, shape=(), dtype=float32)


In [193]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.mean_squared_error #MSE_WithReg(mnist_model, tf.constant(0.01))
loss_history = []

In [194]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logists = mnist_model(images, training=True)
        loss_values = loss_object(labels, logists) 
    
    loss_history.append(loss_value.numpy().mean())
    tf.debugging.assert_equal(logits.shape, (32, 10))
    grads = tape.gradient(loss_values, mnist_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
        

In [198]:
def train(epochs):
    for epoch in range(epochs):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(images, labels)
        print(f"Epoch: {epoch}")

In [199]:
train(3)

InvalidArgumentError: Incompatible shapes: [32,10] vs. [32] [Op:SquaredDifference]

In [139]:
import six
isinstance("", six.string_types)

True